In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from __init__ import *
import numpy as np
from export.saver import read_pickle, save_pickle
from read.svg_reader import SVGReader
from helpers.helpers import filter_none
from placement2.attract import *
from visuals.plotter import plot_general

In [3]:
case  = "amber_a_f01.svg"
sv = SVGReader(case)
sv.run()
# ag = AdjacencyGenerator(sv.layout, buffer_size=0.17)
# ag.run() # TODO shouldnt run if sv.layout is empty
# ag.draw_graph()
domains = sv.layout.domains


In [10]:

plot_general(sv.layout.domains)


In [24]:
print_adjacent_nodes(domains["transit_space"], domains)

[['bath'], ['bath', 'dining'], ['bath', 'dining', 'm_bedroom'], ['bath', 'dining', 'm_bedroom', 'm_closet']]


['bath', 'dining']

In [6]:
get_possible_nbs_east(domains["dining"], domains)

[Domain(closet, x=[8.93-9.93], y=[-0.05-2.36]),
 Domain(m_closet, x=[8.93-9.93], y=[2.69-3.29])]

In [7]:
_, ranges = find_possible_east_nbs_and_ranges(domains["dining"], domains)
ranges

{'closet': [8.78-8.93], 'm_closet': [8.78-8.93]}

In [12]:
poss_nbs = list(ranges.keys())
poss_nbs

['closet', 'm_closet']

In [17]:
comparisons = []
for ix, nb in enumerate(poss_nbs):
    print(ix, nb)
    print([poss_nbs[:ix+1]])
    comparisons.extend([poss_nbs[:ix]])
comparisons

0 closet
[['closet']]
1 m_closet
[['closet', 'm_closet']]


[[], ['closet']]

In [8]:
create_comparisons(list(ranges.keys()))

[['closet']]

In [5]:
curr_dom = domains["bedroom_1"]

poss_nbs = get_possible_nbs_east(curr_dom, domains)
poss_nbs


# for dom in other_doms:
#     if curr_dom.y.line_string_y.intersection(dom.y.line_string_y):
#         print(dom.name)

[Domain(m_bedroom, x=[7.80-9.93], y=[3.35-6.09]),
 Domain(m_bath, x=[5.30-7.62], y=[4.79-6.09]),
 Domain(m_closet, x=[8.93-9.93], y=[2.69-3.29]),
 Domain(bath, x=[5.30-7.62], y=[3.35-4.58]),
 Domain(transit_space, x=[3.36-5.16], y=[2.41-4.19]),
 Domain(laundry, x=[3.38-4.98], y=[4.31-6.06])]

In [71]:
get_possible_nbs_east(domains["laundry"], domains)

[Domain(m_bath, x=[5.30-7.62], y=[4.79-6.09]),
 Domain(bath, x=[5.30-7.62], y=[3.35-4.58]),
 Domain(m_bedroom, x=[7.80-9.93], y=[3.35-6.09])]

In [6]:
sorted_nbs = sorted(poss_nbs, key=lambda n: n.x.min)
sorted_nbs


[Domain(transit_space, x=[3.36-5.16], y=[2.41-4.19]),
 Domain(laundry, x=[3.38-4.98], y=[4.31-6.06]),
 Domain(m_bath, x=[5.30-7.62], y=[4.79-6.09]),
 Domain(bath, x=[5.30-7.62], y=[3.35-4.58]),
 Domain(m_bedroom, x=[7.80-9.93], y=[3.35-6.09]),
 Domain(m_closet, x=[8.93-9.93], y=[2.69-3.29])]

In [46]:
def create_dif_range_x(node:Domain, other:Domain):
    return Range(node.x.max, other.x.min)

ranges = {i.name: create_dif_range_x(curr_dom, i) for i in sorted_nbs}
ranges

{'transit_space': [3.22-3.36],
 'laundry': [3.22-3.38],
 'm_bath': [3.22-5.30],
 'bath': [3.22-5.30],
 'm_bedroom': [3.22-7.80],
 'm_closet': [3.22-8.93]}

In [47]:
def create_comparisons_x(sorted_keys):
    final = []
    for ix, _ in enumerate(sorted_keys):
        final.extend([sorted_keys[:ix]])

    return filter_none(final)

In [50]:
comps  = create_comparisons_x(list(ranges.keys()))
comps

[['transit_space'],
 ['transit_space', 'laundry'],
 ['transit_space', 'laundry', 'm_bath'],
 ['transit_space', 'laundry', 'm_bath', 'bath'],
 ['transit_space', 'laundry', 'm_bath', 'bath', 'm_bedroom']]

In [68]:
comp = comps[1]

def has_intermediate_nodes(comp: list[str]):
    if len(comp) == 1:
        return
    *others, current = comp
    for other in others:
        if ranges[current].contains(domains[other].x):
            return True
        


In [69]:
for comp in comps:
    print(has_intermediate_nodes(comp), comp[-1])
    # stop once we get a True..

None transit_space
None laundry
True m_bath
True bath
True m_bedroom
